# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

# Queries

## FPD0

## SIL

## Alpha - CIC-SIL-Model

cic_model_sil

### Test

In [2]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score score,
  'cic_model_sil_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2369869,af7308b2-0d57-4625-84d5-417f231f55cc,60823698690015,cic_model_sil,0.093818405813199,cic_model_sil_score,2025-10-30 12:08:51,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Appliance,v1,ALL
1,2416425,71b1be7c-53af-4bf2-a04d-aaf1e595388f,60824164250011,cic_model_sil,0.06313448154987157,cic_model_sil_score,2025-10-30 11:00:06,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,SIL Competitor,Appliance,v1,ALL
2,2550427,3f7a0aa9-c819-4c4a-8c2c-0c73fc9b5f57,60825504270014,cic_model_sil,0.09433710952939378,cic_model_sil_score,2025-10-30 12:43:49,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Appliance,v1,ALL
3,2662831,8465fc02-9883-4395-be5b-5a70d9cb9436,60826628310011,cic_model_sil,0.13780941514000333,cic_model_sil_score,2025-10-30 11:15:47,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,SIL Competitor,Appliance,v1,ALL
4,2669305,f5d63cac-3868-4c57-8baf-70d0db22ceca,60826693050012,cic_model_sil,0.18676675488301117,cic_model_sil_score,2025-10-30 11:29:23,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Mall,v1,ALL


In [3]:
df1 = dfd.copy()

### Train

In [4]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score score,
  'cic_model_sil_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2438609,6e3baf27-42c4-4d7a-a71b-32560f668210,60824386090014,Alpha - CIC-SIL-Model,0.207072,cic_model_sil_score,2024-03-23 18:25:44,2024-03-23,2024-03,2024-03-18,2024-03,2024-03-18,Train,1,1,SIL-Instore,Appliance,v1,ALL
1,2476648,2e341ac2-bf1f-4008-bc58-c10b32f2de21,60824766480013,Alpha - CIC-SIL-Model,0.049633,cic_model_sil_score,2024-04-22 15:24:03,2024-04-22,2024-04,2024-04-22,2024-04,2024-04-22,Train,0,1,SIL-Instore,Appliance,v1,ALL
2,2346064,fca72540-9f32-4378-b812-61dcebade28f,60823460640014,Alpha - CIC-SIL-Model,0.157741,cic_model_sil_score,2023-12-24 13:30:48,2023-12-24,2023-12,2023-12-18,2023-12,2023-12-18,Train,1,1,SIL-Instore,Appliance,v1,ALL
3,2419870,3b9a294a-58f8-49e2-9e9f-d4d0a5b83d9c,60824198700019,Alpha - CIC-SIL-Model,0.063305,cic_model_sil_score,2024-03-05 13:24:18,2024-03-05,2024-03,2024-03-04,2024-03,2024-03-04,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2435487,03a2ab53-2a8e-462f-a28f-d6301023b326,60824354870011,Alpha - CIC-SIL-Model,0.074892,cic_model_sil_score,2024-03-20 16:33:06,2024-03-20,2024-03,2024-03-18,2024-03,2024-03-18,Train,0,1,SIL-Instore,Appliance,v1,ALL


In [5]:
df2 = dfd.copy()

In [6]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339659 entries, 0 to 339658
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             339659 non-null  object        
 1   digitalLoanAccountId   339659 non-null  object        
 2   loanAccountNumber      339659 non-null  object        
 3   modelDisplayName       339659 non-null  object        
 4   score                  339659 non-null  object        
 5   score_name             339659 non-null  object        
 6   appln_submit_datetime  339659 non-null  datetime64[us]
 7   disbursementdate       339659 non-null  dbdate        
 8   Disbursement_month     339659 non-null  object        
 9   disbursement_week      339659 non-null  datetime64[us]
 10  Application_month      339659 non-null  object        
 11  application_week       339659 non-null  datetime64[us]
 12  Data_selection         339659 non-null  obje

In [7]:
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')

In [8]:
cicmodelsilfpd0 = df_concat.copy()

In [9]:
cicmodelsilfpd0.groupby(
    ['modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


modelDisplayName       score_name           Disbursement_month
Alpha - CIC-SIL-Model  cic_model_sil_score  2023-01                 185
                                            2023-02                 194
                                            2023-03                 231
                                            2023-04                 283
                                            2023-05                 288
                                            2023-06                 309
                                            2023-07                 320
                                            2023-08                 708
                                            2023-09                1187
                                            2023-10                1385
                                            2023-11                1875
                                            2023-12                3413
                                            2024-01                1807
 

##  alpha_stack_model_sil


### Test

In [10]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score score,
  'alpha_stack_model_sil_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3494871,ee53993c-9a38-4ca4-b227-f5079f23ef42,60834948710017,alpha_stack_model_sil,0.10241822168590894,alpha_stack_model_sil_score,2025-06-13 15:34:40,2025-06-13,2025-06,2025-06-09,2025-06,2025-06-09,Test,0,1,SIL-Instore,Appliance,v1,ALL
1,3501304,3596388c-6eb4-490c-b351-c00ef5d62729,60835013040011,alpha_stack_model_sil,0.17929636429099471,alpha_stack_model_sil_score,2025-06-16 15:31:11,2025-06-16,2025-06,2025-06-16,2025-06,2025-06-16,Test,1,1,SIL-Instore,Appliance,v1,ALL
2,3784469,4fa8866d-f9f7-46e2-b350-cf32234d77b8,60837844690014,alpha_stack_model_sil,0.06301507840341813,alpha_stack_model_sil_score,2025-11-01 15:07:21,2025-11-01,2025-11,2025-10-27,2025-11,2025-10-27,Test,0,1,SIL-Instore,Appliance,v1,ALL
3,3347246,9b806dc0-aab6-4e4e-ad83-998b103b6ebd,60833472460019,alpha_stack_model_sil,0.010246952080656055,alpha_stack_model_sil_score,2025-03-29 15:51:41,2025-03-29,2025-03,2025-03-24,2025-03,2025-03-24,Test,1,1,SIL-Instore,Appliance,v1,ALL
4,3483586,bc9a751b-18c7-4ad8-933e-4f99e25c974b,60834835860016,alpha_stack_model_sil,0.13456821332231547,alpha_stack_model_sil_score,2025-06-07 17:20:03,2025-06-07,2025-06,2025-06-02,2025-06,2025-06-02,Test,0,1,SIL-Instore,Appliance,v1,ALL


In [11]:
df1 = dfd.copy()

### Train

In [12]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Sil_Alpha_Stack_score score,
  'alpha_stack_model_sil_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3257575,12cc40e6-3e15-4bdd-afbe-867cab0f5e68,60832575750012,alpha_stack_model_sil,0.486529,alpha_stack_model_sil_score,2025-10-13 13:12:55,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
1,3041519,1676476c-8518-4be2-805d-ae499d20988b,60830415190013,alpha_stack_model_sil,0.305363,alpha_stack_model_sil_score,2025-10-12 18:04:02,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-06,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
2,3296703,36efd0b7-38d1-4f0f-977a-8d778da72812,60832967030018,alpha_stack_model_sil,0.562340,alpha_stack_model_sil_score,2025-10-13 13:20:44,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
3,3670305,7cb4abea-d929-4454-a7f3-8f75f8198e98,60836703050018,alpha_stack_model_sil,0.653276,alpha_stack_model_sil_score,2025-10-13 17:18:57,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL ZERO,Appliance,v2,Trench 2
4,2234882,a7410136-7e32-4aa3-b6f9-605e50a5f274,60822348820011,alpha_stack_model_sil,0.512194,alpha_stack_model_sil_score,2025-10-10 10:30:35,2025-10-15,2025-10,2025-10-13,2025-10,2025-10-06,Train,1,1,SIL-Instore,Appliance,v2,Trench 2


In [13]:
df2 = dfd.copy()

In [14]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344646 entries, 0 to 344645
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             344646 non-null  object        
 1   digitalLoanAccountId   344646 non-null  object        
 2   loanAccountNumber      344646 non-null  object        
 3   modelDisplayName       344646 non-null  object        
 4   score                  344646 non-null  float64       
 5   score_name             344646 non-null  object        
 6   appln_submit_datetime  344646 non-null  datetime64[us]
 7   disbursementdate       344646 non-null  dbdate        
 8   Disbursement_month     344646 non-null  object        
 9   disbursement_week      344646 non-null  datetime64[us]
 10  Application_month      344646 non-null  object        
 11  application_week       344646 non-null  datetime64[us]
 12  Data_selection         344646 non-null  obje

In [15]:
alphastackmodelsilfpd0 = df_concat.copy()

In [16]:
alphastackmodelsilfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName       score_name                   Disbursement_month
Train           Alpha - StackingModel  alpha_stack_model_sil_score  2023-01                 185
                                                                    2023-02                 194
                                                                    2023-03                 231
                                                                    2023-04                 283
                                                                    2023-05                 288
                                                                    2023-06                 309
                                                                    2023-07                 320
                                                                    2023-08                 708
                                                                    2023-09                1187
                                                 

## Beta SIL STACK Score Model

### Test

In [17]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
            case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score,
  modelDisplayName,
  modelVersionId,
  trenchCategory,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_stack_score score,
  'sil_beta_stack_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3618094,016a3e87-8ad9-4e80-85c9-31599a73a74c,60836180940018,beta_stack_model_sil,0.14161855988029856,sil_beta_stack_score,2025-08-13 13:33:34,2025-08-13,2025-08,2025-08-11,2025-08,2025-08-11,Test,0,1,SIL Competitor,Mall,v1,ALL
1,3618147,01b859d5-0282-49f1-971c-4321afecdaed,60836181470014,beta_stack_model_sil,0.09585902767970964,sil_beta_stack_score,2025-08-13 13:52:51,2025-08-13,2025-08,2025-08-11,2025-08,2025-08-11,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3555511,043b5ee6-658a-453d-86fb-5d11bc95616f,60835555110016,beta_stack_model_sil,0.032122165933676346,sil_beta_stack_score,2025-08-13 13:20:11,2025-08-13,2025-08,2025-08-11,2025-08,2025-08-11,Test,1,1,SIL-Instore,Mall,v1,ALL
3,2501376,04a01142-7302-47e9-bec3-cb0465e7b831,60825013760035,beta_stack_model_sil,0.1554609160680532,sil_beta_stack_score,2025-08-13 13:36:10,2025-08-13,2025-08,2025-08-11,2025-08,2025-08-11,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3618138,068db86c-b379-4cdc-893a-514d84f2a392,60836181380013,beta_stack_model_sil,0.09906880635824285,sil_beta_stack_score,2025-08-13 14:05:57,2025-08-13,2025-08,2025-08-11,2025-08,2025-08-11,Test,0,1,SIL-Instore,Appliance,v1,ALL


In [18]:
df1 = dfd.copy()

### Train

In [19]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
            case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score,
  modelDisplayName,
  modelVersionId,
  trenchCategory,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_stack_score score,
  'sil_beta_stack_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3257575,12cc40e6-3e15-4bdd-afbe-867cab0f5e68,60832575750012,beta_stack_model_sil,0.581567,sil_beta_stack_score,2025-10-13 13:12:55,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
1,3041519,1676476c-8518-4be2-805d-ae499d20988b,60830415190013,beta_stack_model_sil,0.207318,sil_beta_stack_score,2025-10-12 18:04:02,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-06,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
2,3296703,36efd0b7-38d1-4f0f-977a-8d778da72812,60832967030018,beta_stack_model_sil,0.610636,sil_beta_stack_score,2025-10-13 13:20:44,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL Competitor,Appliance,v2,Trench 2
3,3670305,7cb4abea-d929-4454-a7f3-8f75f8198e98,60836703050018,beta_stack_model_sil,0.582057,sil_beta_stack_score,2025-10-13 17:18:57,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL ZERO,Appliance,v2,Trench 2
4,3620838,b8e8581a-e866-4461-a111-f14edb755d69,60836208380017,beta_stack_model_sil,0.692308,sil_beta_stack_score,2025-10-13 18:57:19,2025-10-13,2025-10,2025-10-13,2025-10,2025-10-13,Train,0,1,SIL-Instore,Appliance,v2,Trench 2


In [20]:
df2 = dfd.copy()

In [21]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503361 entries, 0 to 503360
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             503361 non-null  object        
 1   digitalLoanAccountId   503361 non-null  object        
 2   loanAccountNumber      503361 non-null  object        
 3   modelDisplayName       503361 non-null  object        
 4   score                  503361 non-null  float64       
 5   score_name             503361 non-null  object        
 6   appln_submit_datetime  503361 non-null  datetime64[us]
 7   disbursementdate       503361 non-null  dbdate        
 8   Disbursement_month     503361 non-null  object        
 9   disbursement_week      503361 non-null  datetime64[us]
 10  Application_month      503361 non-null  object        
 11  application_week       503361 non-null  datetime64[us]
 12  Data_selection         503361 non-null  obje

In [22]:
betastackmodelsilfpd0 = df_concat.copy()

In [23]:
betastackmodelsilfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName        score_name            Disbursement_month
Train           Beta - StackScoreModel  sil_beta_stack_score  2023-01                 709
                                                              2023-02                 532
                                                              2023-03                 633
                                                              2023-04                 727
                                                              2023-05                 783
                                                              2023-06                 809
                                                              2023-07                1244
                                                              2023-08                2155
                                                              2023-09                3056
                                                              2023-10                4064
                   

# Beta Sil App Score

### Test

In [24]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    case when trenchCategory is null then 'ALL'
 when trenchCategory = '' then 'ALL'
 else trenchCategory end as trenchCategory,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
   ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil'
       else modelDisplayName end as modelDisplayName,
  modelVersionId,
  trenchCategory,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_app_score score,
  'sil_beta_app_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3663711,819ddb25-a4cb-451c-acd0-552066af10c0,60836637110015,apps_score_model_sil,0.447175,sil_beta_app_score,2025-09-04 15:09:30,2025-09-04,2025-09,2025-09-01,2025-09,2025-09-01,Test,0,1,SIL-Instore,Appliance,v1,ALL
1,3494793,f985e70c-1e26-4e70-ba99-8cc71d77591f,60834947930011,apps_score_model_sil,0.559082,sil_beta_app_score,2025-06-13 14:56:32,2025-06-13,2025-06,2025-06-09,2025-06,2025-06-09,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3491345,378f9d1f-496f-4b94-b3ea-2ea59823a308,60834913450018,apps_score_model_sil,0.587126,sil_beta_app_score,2025-06-11 17:18:34,2025-06-11,2025-06,2025-06-09,2025-06,2025-06-09,Test,0,1,SIL-Instore,Appliance,v1,ALL
3,3483852,5464fac7-c1f4-4640-87e7-6e59745142ce,60834838520015,apps_score_model_sil,0.511043,sil_beta_app_score,2025-06-07 18:26:32,2025-06-07,2025-06,2025-06-02,2025-06,2025-06-02,Test,1,1,SIL-Instore,Appliance,v1,ALL
4,3414623,22b478dc-0fbd-4609-84ab-733eb69d4722,60834146230013,apps_score_model_sil,0.492120,sil_beta_app_score,2025-05-02 11:05:56,2025-05-02,2025-05,2025-04-28,2025-05,2025-04-28,Test,1,1,SIL-Instore,Appliance,v1,ALL


In [25]:
df1 = dfd.copy()

### Train

In [26]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
        case when trenchCategory is null then 'ALL'
 when trenchCategory = '' then 'ALL'
 else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
    ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil'
       else modelDisplayName end as modelDisplayName,
  modelVersionId,
  trenchCategory
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_app_score score,
  'sil_beta_app_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2462121,8023789b-63dd-4c0b-9fd5-10e84ba5f466,60824621210016,apps_score_model_sil,0.429673,sil_beta_app_score,2024-04-12 16:49:12,2024-04-12,2024-04,2024-04-08,2024-04,2024-04-08,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2457687,ef0e94d2-c2a6-4a85-a596-a795d22ecf3e,60824576870014,apps_score_model_sil,0.215493,sil_beta_app_score,2024-04-09 12:12:05,2024-04-09,2024-04,2024-04-08,2024-04,2024-04-08,Train,0,1,SIL-Instore,Appliance,v1,ALL
2,2454811,b9f3781c-7d33-4189-a12a-fe9fe830810f,60824548110018,apps_score_model_sil,0.469478,sil_beta_app_score,2024-04-07 10:40:43,2024-04-07,2024-04,2024-04-01,2024-04,2024-04-01,Train,0,1,SIL-Instore,Appliance,v1,ALL
3,2454885,10dfa528-1bec-4e7a-a900-c2eb7359b1ab,60824548850016,apps_score_model_sil,0.569895,sil_beta_app_score,2024-04-07 11:33:45,2024-04-07,2024-04,2024-04-01,2024-04,2024-04-01,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2452447,56a8124c-190e-4ce8-8b43-c262d581562f,60824524470011,apps_score_model_sil,0.559909,sil_beta_app_score,2024-04-05 11:14:34,2024-04-05,2024-04,2024-04-01,2024-04,2024-04-01,Train,0,1,SIL-Instore,Appliance,v1,ALL


In [27]:
df2 = dfd.copy()

In [28]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437428 entries, 0 to 437427
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             437428 non-null  object        
 1   digitalLoanAccountId   437428 non-null  object        
 2   loanAccountNumber      437428 non-null  object        
 3   modelDisplayName       437428 non-null  object        
 4   score                  437428 non-null  float64       
 5   score_name             437428 non-null  object        
 6   appln_submit_datetime  437428 non-null  datetime64[us]
 7   disbursementdate       437428 non-null  dbdate        
 8   Disbursement_month     437428 non-null  object        
 9   disbursement_week      437428 non-null  datetime64[us]
 10  Application_month      437428 non-null  object        
 11  application_week       437428 non-null  datetime64[us]
 12  Data_selection         437428 non-null  obje

In [29]:
appsscoremodelsilfpd0 = df_concat.copy()

In [30]:
appsscoremodelsilfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName      score_name          Disbursement_month
Train           apps_score_model_sil  sil_beta_app_score  2023-01                 622
                                                          2023-02                 462
                                                          2023-03                 523
                                                          2023-04                 625
                                                          2023-05                 665
                                                          2023-06                 683
                                                          2023-07                1035
                                                          2023-08                1767
                                                          2023-09                2591
                                                          2023-10                3579
                                                          2023-

# Beta SIL Demo Score

## Test

In [31]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then 'ALL'
 when trenchCategory = '' then 'ALL'
 else trenchCategory end as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score score,
  'sil_beta_demo_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3644719,19ca4517-960d-46bc-8398-84c8c2a23494,60836447190017,beta_demo_model_sil,0.0872542194,sil_beta_demo_score,2025-08-26 14:43:50,2025-08-26,2025-08,2025-08-25,2025-08,2025-08-25,Test,0,1,SIL Competitor,Appliance,v1,ALL
1,3645979,ba2a3c5d-8a9f-4f2d-9925-b3cc26db41fa,60836459790019,beta_demo_model_sil,0.0512311087,sil_beta_demo_score,2025-08-27 11:52:14,2025-08-27,2025-08,2025-08-25,2025-08,2025-08-25,Test,0,1,SIL-Instore,Appliance,v1,ALL
2,3702282,598ac388-dff1-459b-a6e9-ccd2334913dc,60837022820017,beta_demo_model_sil,0.0332571805,sil_beta_demo_score,2025-09-24 12:02:50,2025-09-24,2025-09,2025-09-22,2025-09,2025-09-22,Test,0,1,SIL Competitor,Appliance,v1,ALL
3,3619735,d4a1bcc9-d6c9-4b66-ad5d-a9c679538e11,60836197350012,beta_demo_model_sil,0.0246658538,sil_beta_demo_score,2025-08-14 11:49:03,2025-08-14,2025-08,2025-08-11,2025-08,2025-08-11,Test,0,1,SIL-Instore,Appliance,v1,ALL
4,3636736,1b3956f8-4f33-41f7-a7e1-20a0bea1dc65,60836367360011,beta_demo_model_sil,0.0973260381,sil_beta_demo_score,2025-08-22 15:06:11,2025-08-22,2025-08,2025-08-18,2025-08,2025-08-18,Test,0,1,SIL-Instore,Mall,v1,ALL


In [32]:
df1 = dfd.copy()

## Train

In [33]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then 'ALL'
 when trenchCategory = '' then 'ALL'
 else trenchCategory end as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score score,
  'sil_beta_demo_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2335066,64798f4e-f30f-459a-81ac-ea28a3ee4abf,60823350660017,beta_demo_model_sil,0.060645,sil_beta_demo_score,2023-12-15 17:43:26,2023-12-15,2023-12,2023-12-11,2023-12,2023-12-11,Train,0,1,SIL-Instore,Appliance,v1,ALL
1,2524925,5a8d7643-6e5f-4c7a-98cc-ecda5e58f0f6,60825249250018,beta_demo_model_sil,0.075890,sil_beta_demo_score,2024-05-20 14:44:44,2024-05-20,2024-05,2024-05-20,2024-05,2024-05-20,Train,1,1,SIL-Instore,Appliance,v1,ALL
2,2356293,468ab4b6-3d56-4b9a-8e2f-1ef4a26b8421,60823562930011,beta_demo_model_sil,0.187466,sil_beta_demo_score,2024-01-02 14:48:30,2024-01-02,2024-01,2024-01-01,2024-01,2024-01-01,Train,0,1,SIL-Instore,Appliance,v1,ALL
3,1988824,c1a8c380-6346-4388-a9a7-654ee6d5dfc8,60819888240025,beta_demo_model_sil,0.099137,sil_beta_demo_score,2023-11-15 18:30:28,2023-11-15,2023-11,2023-11-13,2023-11,2023-11-13,Train,0,1,SIL-Instore,Appliance,v1,ALL
4,2263734,44fd0d7c-aa7b-4b05-8482-804a1f35faac,60822637340011,beta_demo_model_sil,0.087573,sil_beta_demo_score,2023-10-07 16:19:26,2023-10-07,2023-10,2023-10-02,2023-10,2023-10-02,Train,1,1,SIL-Instore,Appliance,v1,ALL


In [34]:
df2 = dfd.copy()

In [35]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504578 entries, 0 to 504577
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             504578 non-null  object        
 1   digitalLoanAccountId   504578 non-null  object        
 2   loanAccountNumber      504578 non-null  object        
 3   modelDisplayName       504578 non-null  object        
 4   score                  504578 non-null  float64       
 5   score_name             504578 non-null  object        
 6   appln_submit_datetime  504578 non-null  datetime64[us]
 7   disbursementdate       504578 non-null  dbdate        
 8   Disbursement_month     504578 non-null  object        
 9   disbursement_week      504578 non-null  datetime64[us]
 10  Application_month      504578 non-null  object        
 11  application_week       504578 non-null  datetime64[us]
 12  Data_selection         504578 non-null  obje

In [36]:
betademomodelsilfpd0 = df_concat.copy()

In [37]:
betademomodelsilfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName     score_name           Disbursement_month
Train           beta_demo_model_sil  sil_beta_demo_score  2023-01                 709
                                                          2023-02                 532
                                                          2023-03                 633
                                                          2023-04                 727
                                                          2023-05                 783
                                                          2023-06                 809
                                                          2023-07                1244
                                                          2023-08                2155
                                                          2023-09                3056
                                                          2023-10                4064
                                                          2023-

# Cash

# Alpha-Cash-CIC-Model

## Test

In [38]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  case when p.modelDisplayName like 'Alpha%' then 'cic_model_cash' else p.modelDisplayName end as modelDisplayName,
  p.modelVersionId,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.aCicScore score,
  'aCicScore' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3315480,68ea2f1d-1770-41ce-b164-46fe7c620d81,60833154800022,cic_model_cash,0.3909224736792525,aCicScore,2025-10-29 17:23:02,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,Quick,not applicable,v1,Trench 3
1,3360136,1b1e7d70-9025-4ff8-a5bf-843a75c66a15,60833601360016,cic_model_cash,0.5767306201598539,aCicScore,2025-10-29 16:21:17,2025-10-29,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,Quick,not applicable,v1,Trench 2
2,3473225,e4775baf-90a2-4fc5-a592-2be6c3fce59f,60834732250019,cic_model_cash,0.4805718809423011,aCicScore,2025-10-29 17:54:37,2025-10-29,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,Quick,not applicable,v1,Trench 2
3,3734817,8ce039b9-81b4-496e-b6af-5aab2a2ba568,60837348170019,cic_model_cash,0.39064570804352716,aCicScore,2025-10-29 18:11:58,2025-10-29,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,Quick,not applicable,v1,Trench 1
4,3769441,554517ec-d636-4f07-83ea-ae56a5957556,60837694410012,cic_model_cash,0.3985980546615801,aCicScore,2025-10-29 18:41:00,2025-10-29,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,Quick,not applicable,v1,Trench 1


In [39]:
df1 = dfd.copy()

## Train

In [40]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  case when modelDisplayName like 'Alpha%' then 'cic_model_cash' else modelDisplayName end as modelDisplayName,
  modelVersionId,
  trenchCategory
  from parsed
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.aCicScore score,
  'aCicScore' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,1687179,3fa4b04d-74b1-4d5d-b998-37f298b01c3e,60816871790035,cic_model_cash,0.671212,aCicScore,2024-09-01 20:25:00,2024-09-01,2024-09,2024-08-26,2024-09,2024-08-26,Train,0,1,Quick,not applicable,v1,Trench 2
1,2815278,db166f16-4715-421e-be7d-a6b47a835d6a,60828152780017,cic_model_cash,0.375778,aCicScore,2024-09-01 17:50:01,2024-09-01,2024-09,2024-08-26,2024-09,2024-08-26,Train,0,1,Quick,not applicable,v1,Trench 1
2,2451785,651c979c-105c-429e-8d28-4573aa0a26b4,60824517850011,cic_model_cash,0.523808,aCicScore,2024-09-01 10:06:14,2024-09-01,2024-09,2024-08-26,2024-09,2024-08-26,Train,0,1,Quick,not applicable,v1,Trench 2
3,2522065,7b45cdf0-0235-425d-9a7f-c89f669d261a,60825220650011,cic_model_cash,0.657538,aCicScore,2024-09-01 19:36:30,2024-09-01,2024-09,2024-08-26,2024-09,2024-08-26,Train,0,1,Quick,not applicable,v1,Trench 2
4,2312617,ba792404-4a6a-4b4b-aae0-3175f9ff1ac8,60823126170022,cic_model_cash,0.406736,aCicScore,2024-09-01 14:06:55,2024-09-01,2024-09,2024-08-26,2024-09,2024-08-26,Train,0,1,Quick,not applicable,v1,Trench 3


In [41]:
df2 = dfd.copy()

In [42]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43340 entries, 0 to 43339
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             43340 non-null  object        
 1   digitalLoanAccountId   43340 non-null  object        
 2   loanAccountNumber      43340 non-null  object        
 3   modelDisplayName       43340 non-null  object        
 4   score                  43340 non-null  float64       
 5   score_name             43340 non-null  object        
 6   appln_submit_datetime  43340 non-null  datetime64[us]
 7   disbursementdate       43340 non-null  dbdate        
 8   Disbursement_month     43340 non-null  object        
 9   disbursement_week      43340 non-null  datetime64[us]
 10  Application_month      43340 non-null  object        
 11  application_week       43340 non-null  datetime64[us]
 12  Data_selection         43340 non-null  object        
 13  d

In [43]:
cicmodelcashfpd0 = df_concat.copy()              

In [44]:
cicmodelcashfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName  score_name  Disbursement_month
Train           cic_model_cash    aCicScore   2024-09               2963
                                              2024-10               3760
                                              2024-11               3665
                                              2024-12               3033
                                              2025-01               3274
                                              2025-02               3128
                                              2025-03               2866
                                              2025-04               2418
                                              2025-05               2721
                                              2025-06               3180
                                              2025-07               3041
                                              2025-08               3131
Test            cic_model_cash    aCicScore   2025-09      

# Alpha-Cash-Stack-Model

## Test

In [45]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  case when p.modelDisplayName like 'Alpha%' then 'alpha_stack_model_cash' else p.modelDisplayName end as modelDisplayName,
  p.modelVersionId,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.aStackScore score,
  'aStackScore' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3486362,988a25a1-676d-4f79-831c-5190216e5f8d,60834863620014,alpha_stack_model_cash,0.3259996755205429,aStackScore,2025-09-27 08:38:21,2025-09-27,2025-09,2025-09-22,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 2
1,3706573,ea6c9e2a-76bc-4202-9771-15000cbb4d0f,60837065730029,alpha_stack_model_cash,0.42176760932635077,aStackScore,2025-09-27 16:03:53,2025-10-06,2025-10,2025-10-06,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 1
2,3708138,7e269de1-887c-44f7-bdb1-088628b49266,60837081380014,alpha_stack_model_cash,0.28516806938442246,aStackScore,2025-09-27 13:54:30,2025-09-27,2025-09,2025-09-22,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 1
3,3427045,399798f3-47e7-4cf6-8c2b-11b4a0afba4e,60834270450018,alpha_stack_model_cash,0.20441350334659414,aStackScore,2025-09-27 14:40:49,2025-10-03,2025-10,2025-09-29,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 2
4,3709317,bb7415c6-63f5-4798-94bd-178a2eddecae,60837093170019,alpha_stack_model_cash,0.2871796963487235,aStackScore,2025-09-27 20:30:38,2025-09-27,2025-09,2025-09-22,2025-09,2025-09-22,Test,1,1,Quick,not applicable,v1,Trench 1


In [46]:
df1 = dfd.copy()

## Train

In [47]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  case when p.modelDisplayName like 'Alpha%' then 'alpha_stack_model_cash' else p.modelDisplayName end as modelDisplayName,
  p.modelVersionId,
  trenchCategory
  from parsed p
  )
  ,
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.aStackScore score,
  'aStackScore' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and flg_mature_fpd0 = 1
  ;
  """

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3704794,3a573575-f3a3-4a2c-9aa2-9ae6d38ab54d,60837047940014,alpha_stack_model_cash,0.410842,aStackScore,2025-09-25 18:01:38,2025-10-07,2025-10,2025-10-06,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 1
1,2709657,b58fa655-02c0-4662-b7e8-25726dc16889,60827096570035,alpha_stack_model_cash,0.381507,aStackScore,2025-09-13 10:13:03,2025-10-11,2025-10,2025-10-06,2025-09,2025-09-08,Train,0,1,Quick,not applicable,v1,Trench 3
2,2633854,322dcc93-8bc8-4f34-a52f-ac80579afc9d,60826338540017,alpha_stack_model_cash,0.265343,aStackScore,2025-09-22 22:25:25,2025-10-15,2025-10,2025-10-13,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 2
3,3707368,40ccc247-d2b1-492e-919c-9ac0f68bac12,60837073680012,alpha_stack_model_cash,0.517128,aStackScore,2025-09-29 12:58:53,2025-10-06,2025-10,2025-10-06,2025-09,2025-09-29,Train,0,1,Quick,not applicable,v1,Trench 1
4,3695217,0658ccea-9c62-45c4-b0fe-7736bf63951c,60836952170019,alpha_stack_model_cash,0.477718,aStackScore,2025-09-20 18:02:14,2025-10-09,2025-10,2025-10-06,2025-09,2025-09-15,Train,1,1,Quick,not applicable,v1,Trench 1


In [48]:
df2 = dfd.copy()

In [49]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39703 entries, 0 to 39702
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             39703 non-null  object        
 1   digitalLoanAccountId   39703 non-null  object        
 2   loanAccountNumber      39703 non-null  object        
 3   modelDisplayName       39703 non-null  object        
 4   score                  39703 non-null  float64       
 5   score_name             39703 non-null  object        
 6   appln_submit_datetime  39703 non-null  datetime64[us]
 7   disbursementdate       39703 non-null  dbdate        
 8   Disbursement_month     39703 non-null  object        
 9   disbursement_week      39703 non-null  datetime64[us]
 10  Application_month      39703 non-null  object        
 11  application_week       39703 non-null  datetime64[us]
 12  Data_selection         39703 non-null  object        
 13  d

In [50]:
alphastackmodelcashfpd0 = df_concat.copy()

In [51]:
alphastackmodelcashfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName        score_name   Disbursement_month
Train           alpha_stack_model_cash  aStackScore  2024-10               3620
                                                     2024-11               3623
                                                     2024-12               2918
                                                     2025-01               3171
                                                     2025-02               3070
                                                     2025-03               2822
                                                     2025-04               2388
                                                     2025-05               2698
                                                     2025-06               3146
                                                     2025-07               3017
                                                     2025-08               3108
Test            alpha_stack_model_cash  aStackSc

# **If we need for all trench category also we will have to create a train and test for all ignoring the trench category.**

# Beta-Cash-Demo-Model

## Test

In [52]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  case when p.modelDisplayName like 'Beta-Cash-Demo-Model' then 'beta_demo_model_cash' else p.modelDisplayName end as modelDisplayName,
  p.modelVersionId
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  ), 
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Beta_Cash_Demo_Score score,
  'Beta_Cash_Demo_Score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3718881,7c1dead3-748f-4299-a825-7e76c7d10c81,60837188810013,beta_demo_model_cash,0.3114746517368309,Beta_Cash_Demo_Score,2025-10-02 15:31:34,2025-10-02,2025-10,2025-09-29,2025-10,2025-09-29,Test,0,1,Quick,not applicable,v1,Trench 1
1,3230169,9ed41d23-400d-4654-b0a7-dbd7e514bdfa,60832301690022,beta_demo_model_cash,0.5116923749333839,Beta_Cash_Demo_Score,2025-10-05 09:38:47,2025-10-05,2025-10,2025-09-29,2025-10,2025-09-29,Test,0,1,Quick,not applicable,v1,Trench 2
2,3708138,7e269de1-887c-44f7-bdb1-088628b49266,60837081380014,beta_demo_model_cash,0.4464579469464231,Beta_Cash_Demo_Score,2025-09-27 13:54:30,2025-09-27,2025-09,2025-09-22,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 1
3,3486362,988a25a1-676d-4f79-831c-5190216e5f8d,60834863620014,beta_demo_model_cash,0.4722916732952999,Beta_Cash_Demo_Score,2025-09-27 08:38:21,2025-09-27,2025-09,2025-09-22,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 2
4,3709389,c7182945-70ff-4091-bccd-47241b8fce48,60837093890015,beta_demo_model_cash,0.5338976464327317,Beta_Cash_Demo_Score,2025-09-27 23:18:25,2025-09-29,2025-09,2025-09-29,2025-09,2025-09-22,Test,0,1,Quick,not applicable,v1,Trench 1


In [53]:
df1 = dfd.copy()

## Train

In [54]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  case when modelDisplayName like 'Beta-Cash-Demo-Model' then 'beta_demo_model_cash' else modelDisplayName end as modelDisplayName,
  modelVersionId,
  trenchCategory
  from parsed
  )
  ,
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Beta_Cash_Demo_Score score,
  'Beta_Cash_Demo_Score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,1705645,20b32a53-2a52-4538-ad77-1a0fc165c8e5,60817056450021,beta_demo_model_cash,0.383484,Beta_Cash_Demo_Score,2025-09-10 06:30:58,2025-10-06,2025-10,2025-10-06,2025-09,2025-09-08,Train,0,1,Quick,not applicable,v1,Trench 2
1,3690579,228a6775-72c1-466a-b0a3-3db05207a132,60836905790012,beta_demo_model_cash,0.572927,Beta_Cash_Demo_Score,2025-09-18 12:03:14,2025-10-17,2025-10,2025-10-13,2025-09,2025-09-15,Train,1,1,Quick,not applicable,v1,Trench 1
2,3706573,ea6c9e2a-76bc-4202-9771-15000cbb4d0f,60837065730029,beta_demo_model_cash,0.457281,Beta_Cash_Demo_Score,2025-09-27 16:03:53,2025-10-06,2025-10,2025-10-06,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 1
3,2899744,939e56ae-3211-41e1-a999-0078620287b7,60828997440026,beta_demo_model_cash,0.385395,Beta_Cash_Demo_Score,2025-09-23 08:16:35,2025-10-15,2025-10,2025-10-13,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 3
4,3680631,009fbcc6-ecc0-4354-b634-4cb4ba2b2149,60836806310014,beta_demo_model_cash,0.502376,Beta_Cash_Demo_Score,2025-09-14 11:30:42,2025-10-15,2025-10,2025-10-13,2025-09,2025-09-08,Train,1,1,Quick,not applicable,v1,Trench 1


In [55]:
df2 = dfd.copy()

In [56]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42075 entries, 0 to 42074
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             42075 non-null  object        
 1   digitalLoanAccountId   42075 non-null  object        
 2   loanAccountNumber      42075 non-null  object        
 3   modelDisplayName       42075 non-null  object        
 4   score                  42075 non-null  float64       
 5   score_name             42075 non-null  object        
 6   appln_submit_datetime  42075 non-null  datetime64[us]
 7   disbursementdate       42075 non-null  dbdate        
 8   Disbursement_month     42075 non-null  object        
 9   disbursement_week      42075 non-null  datetime64[us]
 10  Application_month      42075 non-null  object        
 11  application_week       42075 non-null  datetime64[us]
 12  Data_selection         42075 non-null  object        
 13  d

In [57]:
betademomodelcashfpd0 = df_concat.copy()

In [58]:
betademomodelcashfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName      score_name            Disbursement_month
Train           beta_demo_model_cash  Beta_Cash_Demo_Score  2024-10               3892
                                                            2024-11               3845
                                                            2024-12               3268
                                                            2025-01               3509
                                                            2025-02               3333
                                                            2025-03               2968
                                                            2025-04               2596
                                                            2025-05               2748
                                                            2025-06               3224
                                                            2025-07               3129
                                                   

# Beta-Cash-AppScore-Model

## Test

In [59]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  case when modelDisplayName like 'Beta-Cash-AppScore-Model' then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
  p.modelVersionId,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.beta_cash_app_score score,
  'beta_cash_app_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3745103,4584199d-d30b-4487-bff9-7c1a6cae1b9d,60837451030011,apps_score_cash,0.409782,beta_cash_app_score,2025-10-15 04:04:48,2025-10-15,2025-10,2025-10-13,2025-10,2025-10-13,Test,1,1,Quick,not applicable,v1,Trench 1
1,3775984,b770cfe8-bd54-46c6-b3c5-a197a405e501,60837759840017,apps_score_cash,0.485792,beta_cash_app_score,2025-10-28 17:48:00,2025-10-29,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,Quick,not applicable,v1,Trench 1
2,3773922,66e92ada-d18e-45b4-ac80-33b097bf7ae7,60837739220015,apps_score_cash,0.498122,beta_cash_app_score,2025-10-27 18:21:08,2025-10-27,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,Quick,not applicable,v1,Trench 1
3,3704456,89a3b123-fd4f-4148-be41-72774d8afca6,60837044560019,apps_score_cash,0.317884,beta_cash_app_score,2025-10-12 14:20:11,2025-10-12,2025-10,2025-10-06,2025-10,2025-10-06,Test,1,1,Quick,not applicable,v1,Trench 1
4,3771768,8ce33f28-b37e-42b5-959d-b9c434f39c53,60837717680015,apps_score_cash,0.451841,beta_cash_app_score,2025-10-26 22:24:08,2025-10-27,2025-10,2025-10-27,2025-10,2025-10-20,Test,0,1,Quick,not applicable,v1,Trench 1


In [60]:
df1 = dfd.copy()

## Train

In [61]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  case when modelDisplayName like 'Beta-Cash-AppScore-Model' then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
  modelVersionId,
  trenchCategory
  from parsed
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.beta_cash_app_score score,
  'beta_cash_app_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2637626,fe5d248d-445e-4512-ba2b-8932dca1d0ba,60826376260013,apps_score_cash,0.671163,beta_cash_app_score,2024-07-06 11:28:50,2024-07-06,2024-07,2024-07-01,2024-07,2024-07-01,Train,0,1,Quick,not applicable,v1,Trench 1
1,1766299,17edd439-10d8-47d4-b074-3878e2be4cd4,60817662990026,apps_score_cash,0.483240,beta_cash_app_score,2023-08-11 10:38:45,2023-08-11,2023-08,2023-08-07,2023-08,2023-08-07,Train,0,1,Flex-up,not applicable,v1,Trench 3
2,1014285,d874c33b-64ad-47fa-99cf-8fcb180cb982,60810142850025,apps_score_cash,0.479285,beta_cash_app_score,2023-11-08 18:44:22,2023-12-18,2023-12,2023-12-18,2023-11,2023-11-06,Train,0,1,Flex-up,not applicable,v1,Trench 3
3,2482105,d4bef281-54c0-4f14-9227-5c605ecdc4c5,60824821050016,apps_score_cash,0.380734,beta_cash_app_score,2024-05-05 16:20:23,2024-05-05,2024-05,2024-04-29,2024-05,2024-04-29,Train,0,1,Quick,not applicable,v1,Trench 1
4,2184507,e0766692-feb0-4bff-9519-7f76a1e4efe9,60821845070028,apps_score_cash,0.539853,beta_cash_app_score,2024-03-19 12:14:01,2024-03-20,2024-03,2024-03-18,2024-03,2024-03-18,Train,0,1,Flex-up,not applicable,v1,Trench 3


In [62]:
df2 = dfd.copy()

In [63]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62462 entries, 0 to 62461
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             62462 non-null  object        
 1   digitalLoanAccountId   62462 non-null  object        
 2   loanAccountNumber      62462 non-null  object        
 3   modelDisplayName       62462 non-null  object        
 4   score                  62462 non-null  float64       
 5   score_name             62462 non-null  object        
 6   appln_submit_datetime  62462 non-null  datetime64[us]
 7   disbursementdate       62462 non-null  dbdate        
 8   Disbursement_month     62462 non-null  object        
 9   disbursement_week      62462 non-null  datetime64[us]
 10  Application_month      62462 non-null  object        
 11  application_week       62462 non-null  datetime64[us]
 12  Data_selection         62462 non-null  object        
 13  d

In [64]:
appsscorecashfpd0 = df_concat.copy()

In [65]:
appsscorecashfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName  score_name           Disbursement_month
Train           apps_score_cash   beta_cash_app_score  2023-01                 77
                                                       2023-02                 71
                                                       2023-03                110
                                                       2023-04                158
                                                       2023-05                202
                                                       2023-06                204
                                                       2023-07                234
                                                       2023-08                229
                                                       2023-09                262
                                                       2023-10                179
                                                       2023-11                261
                        

# Beta-Cash-Stack-Model

## Test

In [66]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  case when modelDisplayName like 'Beta-Cash-Stack-Model' then 'beta_stack_model_cash' else p.modelDisplayName end as modelDisplayName,
  p.modelVersionId
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Beta_cash_stack_score score,
  'Beta_cash_stack_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,3787034,fdc1c6f6-aa8d-4028-bcd4-9833503cc83c,60837870340013,beta_stack_model_cash,0.5023654828786341,Beta_cash_stack_score,2025-11-02 16:52:42,2025-11-03,2025-11,2025-11-03,2025-11,2025-10-27,Test,0,1,Quick,not applicable,v1,Trench 1
1,3013833,56a168f7-10d4-4e39-b4cc-84cd936fd98c,60830138330021,beta_stack_model_cash,0.6631651547338252,Beta_cash_stack_score,2025-09-28 18:37:35,2025-09-28,2025-09,2025-09-22,2025-09,2025-09-22,Test,1,1,Quick,not applicable,v1,Trench 3
2,2863259,78fce405-fd35-4832-ab50-eb15cbf54b89,60828632590029,beta_stack_model_cash,0.45845159697816207,Beta_cash_stack_score,2025-10-02 10:17:03,2025-10-02,2025-10,2025-09-29,2025-10,2025-09-29,Test,0,1,Quick,not applicable,v1,Trench 3
3,1080796,27ed2366-6b6c-4c53-b984-ef57da6466bd,60810807960064,beta_stack_model_cash,0.18047019040354925,Beta_cash_stack_score,2025-10-02 01:05:15,2025-10-02,2025-10,2025-09-29,2025-10,2025-09-29,Test,0,1,Quick,not applicable,v1,Trench 3
4,3357515,58f0e9c2-1805-4158-a3d6-c8f5759aa2af,60833575150024,beta_stack_model_cash,0.373608810118396,Beta_cash_stack_score,2025-10-02 16:13:23,2025-10-02,2025-10,2025-09-29,2025-10,2025-09-29,Test,0,1,Quick,not applicable,v1,Trench 3


In [67]:
df1 = dfd.copy()

## Train

In [68]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
    case when modelDisplayName like 'Beta-Cash-Stack-Model' then 'beta_stack_model_cash' else modelDisplayName end as modelDisplayName,
    modelVersionId,
  trenchCategory
  from parsed
  )
,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Beta_cash_stack_score score,
  'Beta_cash_stack_score' score_name,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', date(loanmaster.disbursementDateTime)) as Disbursement_month,
  DATE_TRUNC(disbursementDateTime, WEEK(MONDAY)) disbursement_week,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  DATE_TRUNC(coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)), WEEK(MONDAY)) application_week,  
  'Train' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
      case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  modelVersionId, trenchCategory,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2690355,c7857817-cf08-456b-a342-281f16388828,60826903550037,beta_stack_model_cash,0.622889,Beta_cash_stack_score,2025-09-25 10:18:13,2025-10-14,2025-10,2025-10-13,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 3
1,2759009,5949fe53-68b1-4976-b609-c81b3e4e7ac1,60827590090018,beta_stack_model_cash,0.171244,Beta_cash_stack_score,2025-09-23 12:36:35,2025-10-16,2025-10,2025-10-13,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 2
2,3707107,3e52021e-5fa8-485d-a7ac-4bd42c5b6ebe,60837071070013,beta_stack_model_cash,0.423079,Beta_cash_stack_score,2025-09-27 02:11:19,2025-10-14,2025-10,2025-10-13,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 1
3,3704794,3a573575-f3a3-4a2c-9aa2-9ae6d38ab54d,60837047940014,beta_stack_model_cash,0.460314,Beta_cash_stack_score,2025-09-25 18:01:38,2025-10-07,2025-10,2025-10-06,2025-09,2025-09-22,Train,0,1,Quick,not applicable,v1,Trench 1
4,2396226,464e081e-9da3-411e-a011-ba0a50bb96b5,60823962260035,beta_stack_model_cash,0.684145,Beta_cash_stack_score,2025-09-30 22:02:41,2025-10-16,2025-10,2025-10-13,2025-09,2025-09-29,Train,0,1,Quick,not applicable,v1,Trench 3


In [69]:
df2 = dfd.copy()

In [70]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat['score'] = pd.to_numeric(df_concat['score'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41457 entries, 0 to 41456
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             41457 non-null  object        
 1   digitalLoanAccountId   41457 non-null  object        
 2   loanAccountNumber      41457 non-null  object        
 3   modelDisplayName       41457 non-null  object        
 4   score                  41457 non-null  float64       
 5   score_name             41457 non-null  object        
 6   appln_submit_datetime  41457 non-null  datetime64[us]
 7   disbursementdate       41457 non-null  dbdate        
 8   Disbursement_month     41457 non-null  object        
 9   disbursement_week      41457 non-null  datetime64[us]
 10  Application_month      41457 non-null  object        
 11  application_week       41457 non-null  datetime64[us]
 12  Data_selection         41457 non-null  object        
 13  d

In [71]:
betastackmodelcashfpd0 = df_concat.copy()

In [72]:
betastackmodelcashfpd0.groupby(
    ['Data_selection','modelDisplayName','score_name','Disbursement_month']
)['flg_mature_fpd0'].sum().sort_index(level='Disbursement_month')


Data_selection  modelDisplayName       score_name             Disbursement_month
Train           beta_stack_model_cash  Beta_cash_stack_score  2024-10               3843
                                                              2024-11               3800
                                                              2024-12               3142
                                                              2025-01               3399
                                                              2025-02               3270
                                                              2025-03               2923
                                                              2025-04               2561
                                                              2025-05               2724
                                                              2025-06               3190
                                                              2025-07               3105
                             

In [73]:
cicmodelsilfpd0.columns.values

array(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'score', 'score_name', 'appln_submit_datetime',
       'disbursementdate', 'Disbursement_month', 'disbursement_week',
       'Application_month', 'application_week', 'Data_selection',
       'deffpd0', 'flg_mature_fpd0', 'new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory'], dtype=object)

In [74]:
import functools
dataframes = [
cicmodelsilfpd0,
alphastackmodelsilfpd0,
betastackmodelsilfpd0,
appsscoremodelsilfpd0,
betademomodelsilfpd0,
cicmodelcashfpd0, 
alphastackmodelcashfpd0,
betademomodelcashfpd0,
appsscorecashfpd0,
betastackmodelcashfpd0]
common_columns = ['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'score', 'score_name', 'appln_submit_datetime',
       'disbursementdate', 'Disbursement_month', 'disbursement_week',
       'Application_month', 'application_week', 'Data_selection',
       'deffpd0', 'flg_mature_fpd0', 'new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'score', 'score_name', 'appln_submit_datetime',
       'disbursementdate', 'Disbursement_month', 'disbursement_week',
       'Application_month', 'application_week', 'Data_selection',
       'deffpd0', 'flg_mature_fpd0', 'new_loan_type', 'loan_product_type',
       'modelVersionId', 'trenchCategory'], dtype=object)

In [75]:
final_df.head()

,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,score,score_name,appln_submit_datetime,disbursementdate,Disbursement_month,disbursement_week,Application_month,application_week,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,loan_product_type,modelVersionId,trenchCategory
0,2369869,af7308b2-0d57-4625-84d5-417f231f55cc,60823698690015,cic_model_sil,0.093818,cic_model_sil_score,2025-10-30 12:08:51,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Appliance,v1,ALL
1,2416425,71b1be7c-53af-4bf2-a04d-aaf1e595388f,60824164250011,cic_model_sil,0.063134,cic_model_sil_score,2025-10-30 11:00:06,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,SIL Competitor,Appliance,v1,ALL
2,2550427,3f7a0aa9-c819-4c4a-8c2c-0c73fc9b5f57,60825504270014,cic_model_sil,0.094337,cic_model_sil_score,2025-10-30 12:43:49,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Appliance,v1,ALL
3,2662831,8465fc02-9883-4395-be5b-5a70d9cb9436,60826628310011,cic_model_sil,0.137809,cic_model_sil_score,2025-10-30 11:15:47,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,1,1,SIL Competitor,Appliance,v1,ALL
4,2669305,f5d63cac-3868-4c57-8baf-70d0db22ceca,60826693050012,cic_model_sil,0.186767,cic_model_sil_score,2025-10-30 11:29:23,2025-10-30,2025-10,2025-10-27,2025-10,2025-10-27,Test,0,1,SIL Competitor,Mall,v1,ALL


In [78]:
# Convert dbdate column to datetime
final_df['disbursementdate'] = pd.to_datetime(final_df['disbursementdate'])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2358709 entries, 0 to 2358708
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   customerId             object        
 1   digitalLoanAccountId   object        
 2   loanAccountNumber      object        
 3   modelDisplayName       object        
 4   score                  float64       
 5   score_name             object        
 6   appln_submit_datetime  datetime64[us]
 7   disbursementdate       datetime64[ns]
 8   Disbursement_month     object        
 9   disbursement_week      datetime64[us]
 10  Application_month      object        
 11  application_week       datetime64[us]
 12  Data_selection         object        
 13  deffpd0                Int64         
 14  flg_mature_fpd0        Int64         
 15  new_loan_type          object        
 16  loan_product_type      object        
 17  modelVersionId         object        
 18  trenchCategory        

In [85]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.work_fpd0_gini_decdev_all_models"

# Convert object columns to string
for col in final_df.columns:
    if final_df[col].dtype == 'object':
        final_df[col] = final_df[col].astype(str)

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    final_df, 
    table_id, 
    job_config=job_config,
    parquet_compression='snappy'
)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4ae8284b-a9ec-4255-8a91-0ed1ea9d10d7>

# End